## Limpeza e harmonização dos dados

<p>Importando bibliotecas</p>

In [1]:
import os

In [2]:
from functools import reduce

In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [4]:
from myfunctions import corrige_data, remove_caracter_special, strings_sg_para_numeros, renomeia_vacina

Definindo pastas de trabalho

In [5]:
local_dir = os.path.dirname('__file__')

In [6]:
output_dir = os.path.join(local_dir, 'curation')

In [7]:
datalake = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE/2022-04-19/'

Criando sessão spark

In [8]:
spark = SparkSession.builder.appName('curation').getOrCreate()

22/05/05 15:51:24 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/05/05 15:51:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 15:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Trabalhando com sindrome gripal

In [9]:
dir_sg = ['sindrome_gripal-2020', 'sindrome_gripal-2021', 'sindrome_gripal-2022']

In [10]:
dfs = [spark.read.csv(os.path.join(datalake, folder), header=True, sep=';') for folder in dir_sg]

In [11]:
sg = reduce(lambda d1, d2: d1.unionAll(d2), dfs)

In [12]:
columns = df.columns

In [13]:
for col in columns[1:]:
    sg = sg.withColumn(col, corrige_data(col))
    sg = sg.withColumn(col, strings_sg_para_numeros(remove_caracter_special(col)))

In [ ]:
sg = sg.withColumn(
    'date',
    F.when(
        F.col('dataInicioSintomas').isNotNull(),
        F.lit(F.col('dataInicioSintomas'))
    ).when(
        F.col('dataInicioSintomas').isNull(),
        F.when(
            F.col('dataColetaTeste4').isNotNull(),
            F.lit(F.col('dataColetaTeste4'))
        ).when(
            F.col('dataColetaTeste4').isNull(),
            F.when(
                F.col('dataColetaTeste3').isNotNull(),
                F.lit(F.col('dataColetaTeste3'))
            ).when(
                F.col('dataColetaTeste3').isNull(),
                F.when(
                    F.col('dataColetaTeste2').isNotNull(),
                    F.lit(F.col('dataColetaTeste2'))
                ).when(
                    F.col('dataColetaTeste2').isNull(),
                    F.lit(F.col('dataColetaTeste1'))
                ).otherwise(F.col('dataNotificacao'))
            )
        )
    )
)

In [ ]:
sg = sg.withColumn(
    'municipioIBGE',
    F.when(
        F.col('municipioIBGE').isNull(),
        F.lit(F.col('municipioNotificacaoIBGE'))
    ).otherwise(F.col('municipioIBGE'))
)

In [ ]:
sg = sg.withColumnRenamed('municipioIBGE', 'code_mun_ibge')

In [ ]:
sg = sg.withColumn(
    'newCases_SG',
    F.when(
        (F.col('classificacaoFinal') == 1) |
        (F.col('classificacaoFinal') == 2) |
        (F.col('lcassificacaoFinal') == 3) |
        (F.col('classificacaoFinal') == 4) |
        (F.col('classificacaoFinal') == 5),
        F.lit(1)
    )
)

sg = sg.withColumn('new_des_SG', F.when(F.col('classificacaoFinal') == 7, F.lit(1)))

sg = sg.withColumn('new_undefined_SG', F.when(F.col('classificacaoFinal').isNull(), F.lit(1)))

In [ ]:
sg = sg.withColumn(
    'newDeath_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 1), F.lit(1)
    )
).withColumn(
    'newDeath_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 1), F.lit(1)
    )
).withColumn(
    'newDeath_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 1), F.lit(1)
    )
)

In [ ]:
sg = sg.withColumn(
    'new_recovered_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 2), F.lit(1)
    )
).withColumn(
    'new_recovered_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 2), F.lit(1)
    )
).withColumn(
    'new_recovered_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 2), F.lit(1)
    )
)

In [ ]:
sg = sg.withColumn(
    'hosp_uti_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 4),
        F.lit(1)
    )
).withColumn(
    'hosp_uti_des_SG',
    F.when(
        (F.col('new_des_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 4),
        F.lit(1)
    )
).withColumn(
    'hosp_uti_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 4),
        F.lit(1)
    )
)

In [ ]:
sg = sg.withColumn(
    'dom_trat_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 6),
        F.lit(1)
    )
).withColumn(
    'dom_trat_des_SG',
    F.when(
        (F.col('new_des_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 6),
        F.lit(1)
    )
).withColumn(
    'dom_trat_undefined_SG',
    F.when(
        (F.col('new_undefined_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 6),
        F.lit(1)
    )
)

In [ ]:
sg = sg.withColumn(
    'hos_clinc_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 3),
        F.lit(1)
    )
).withColumn(
    'hos_clinc_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 3),
        F.lit(1)
    )
).withColumn(
    'hos_clinc_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 3),
        F.lit(1)
    )
)

In [ ]:
new_columns = [
    'newCases_SG', 'new_des_SG', 'new_undefined_SG',
    'newDeath_SG', 'newDeath_des_SG', 'newDeath_undefined_SG',
    'new_recovered_SG', 'new_recovered_des_SG', 'new_recovered_undefined_SG',
    'hosp_uti_SG', 'hosp_uti_des_SG', 'hosp_uti_undefined_SG',
    'dom_trat_SG', 'dom_trat_des_SG', 'dom_trat_undefined_SG',
    'hos_clinc_SG', 'hos_clinc_des_SG', 'hos_clinc_undefined_SG',

]

In [ ]:
df.groupby('date', 'code_mun_ibge').agg(
    *[F.count(col).alias(col) for col in new_columns]
).filter(F.col('date') >= '2020-01-01').orderBy('date') \
.write.csv(os.path.join(output_dir, 'sindrome_gripal'), header=True, mode='overwrite')

### Vacinação

In [9]:
vac = spark.read.csv(os.path.join(datalake, 'vacinacao'), header=True, sep=';')

In [10]:
vac.show(1, truncate=False, vertical=True)

22/05/05 15:51:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0----------------------------------------------------------------------------------------------------
 document_id                              | 00462514-0d0e-4082-807c-1b7868a02367-i0b0                        
 paciente_id                              | 43f9ad3984ca1276c0557d43af4fd66a285dc907a76c9324b911d42029500250 
 paciente_idade                           | 60                                                               
 paciente_dataNascimento                  | 1960-11-12                                                       
 paciente_enumSexoBiologico               | M                                                                
 paciente_racaCor_codigo                  | 02                                                               
 paciente_racaCor_valor                   | PRETA                                                            
 paciente_endereco_coIbgeMunicipio        | 410690                                                           
 paciente_

In [11]:
for col in vac.columns[2:]:
    vac = vac.withColumn(col, corrige_data(col))
    vac = vac.withColumn(col, strings_sg_para_numeros(remove_caracter_special(col)))
vac = vac.withColumn('vacina_nome', renomeia_vacina('vacina_nome'))

In [15]:
vac = vac.withColumn('dt_pattern', F.lit('2022-04-19'))

In [12]:
vac.show(1, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------
 document_id                              | 00462514-0d0e-4082-807c-1b7868a02367-i0b0                        
 paciente_id                              | 43f9ad3984ca1276c0557d43af4fd66a285dc907a76c9324b911d42029500250 
 paciente_idade                           | 60                                                               
 paciente_dataNascimento                  | 1960-11-12                                                       
 paciente_enumSexoBiologico               | m                                                                
 paciente_racaCor_codigo                  | 02                                                               
 paciente_racaCor_valor                   | 2                                                                
 paciente_endereco_coIbgeMunicipio        | 410690                                                           
 paciente_

In [ ]:
vac = vac.withColumn(
    'num_dose_reg_VAC',
    F.when(
        (F.col('1a dose').isNotNull()) |
        (F.col('1o reforco').isNotNull()) |
        (F.col('2a dose').isNotNull()) |
        (F.col('2o reforco').isNotNull()) |
        (F.col('3a dose').isNotNull()) |
        (F.col('3o reforco').isNotNull()) |
        (F.col('4a dose').isNotNull()) |
        (F.col('5a dose').isNotNull()) |
        (F.col('revacinacao').isNotNull()) |
        (F.col('1a dose revacinacao').isNotNull()) |
        (F.col('2a dose revacinacao').isNotNull()) |
        (F.col('3a dose revacinacao').isNotNull()) |
        (F.col('dose inicial').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('reforco').isNotNull()) |
        (F.col('unica').isNotNull()) |
        (F.col('tratamento com uma dose').isNotNull()) |
        (F.col('tratamento com dezessete doses').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_pri_VAC',
    F.when(
        ((F.col('1a dose').isNotNull()) |
        (F.col('dose inicial').isNotNull())) &
        (F.col('2a dose').isNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_sec_uni_VAC',
    F.when(
        (F.col('2a dose').isNotNull()) |
        (F.col('unica').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_reinforcment_VAC',
    F.when(
        (F.col('3a dose').isNotNull()) |
        (F.col('4a dose').isNotNull()) |
        (F.col('5a dose').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('1a dose revacinacao').isNotNull()) |
        (F.col('2a dose revacinacao').isNotNull()) |
        (F.col('3a dose revacinacao').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('reforco').isNotNull()) |
        (F.col('1o reforco').isNotNull()) |
        (F.col('2o reforco').isNotNull()) |
        (F.col('3o reforco').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_non_reg_VAC',
    F.when(F.col('dose').isNotNull(), F.lit(1))
).withColumn(
    'total_dose_VAC',
    F.col('num_dose_reg_VAC')+F.col('num_dose_non_reg_VAC')
)

In [ ]:
vac = vac.withColumn(
    'num_vac_type_1_VAC',
    F.when(
        (F.col('1a dose') == 1) |
        (F.col('1o reforco') == 1) |
        (F.col('2a dose') == 1) |
        (F.col('2o reforco') == 1) |
        (F.col('3a dose') == 1) |
        (F.col('3o reforco') == 1) |
        (F.col('4a dose') == 1) |
        (F.col('5a dose') == 1) |
        (F.col('revacinacao') == 1) |
        (F.col('1a dose revacinacao') == 1) |
        (F.col('2a dose revacinacao') == 1) |
        (F.col('3a dose revacinacao') == 1) |
        (F.col('dose inicial') == 1) |
        (F.col('dose adicional') == 1) |
        (F.col('reforco') == 1) |
        (F.col('unica') == 1) |
        (F.col('tratamento com uma dose') == 1) |
        (F.col('tratamento com dezessete doses') == 1),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_2_VAC',
    F.when(
        (F.col('1a dose') == 2) |
        (F.col('1o reforco') == 2) |
        (F.col('2a dose') == 2) |
        (F.col('2o reforco') == 2) |
        (F.col('3a dose') == 2) |
        (F.col('3o reforco') == 2) |
        (F.col('4a dose') == 2) |
        (F.col('5a dose') == 2) |
        (F.col('revacinacao') == 2) |
        (F.col('1a dose revacinacao') == 2) |
        (F.col('2a dose revacinacao') == 2) |
        (F.col('3a dose revacinacao') == 2) |
        (F.col('dose inicial') == 2) |
        (F.col('dose adicional') == 2) |
        (F.col('reforco') == 2) |
        (F.col('unica') == 2) |
        (F.col('tratamento com uma dose') == 2) |
        (F.col('tratamento com dezessete doses') == 2),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_3_VAC',
    F.when(
        (F.col('1a dose') == 3) |
        (F.col('1o reforco') == 3) |
        (F.col('2a dose') == 3) |
        (F.col('2o reforco') == 3) |
        (F.col('3a dose') == 3) |
        (F.col('3o reforco') == 3) |
        (F.col('4a dose') == 3) |
        (F.col('5a dose') == 3) |
        (F.col('revacinacao') == 3) |
        (F.col('1a dose revacinacao') == 3) |
        (F.col('2a dose revacinacao') == 3) |
        (F.col('3a dose revacinacao') == 3) |
        (F.col('dose inicial') == 3) |
        (F.col('dose adicional') == 3) |
        (F.col('reforco') == 3) |
        (F.col('unica') == 3) |
        (F.col('tratamento com uma dose') == 3) |
        (F.col('tratamento com dezessete doses') == 3),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_4_VAC',
    F.when(
        (F.col('1a dose') == 4) |
        (F.col('1o reforco') == 4) |
        (F.col('2a dose') == 4) |
        (F.col('2o reforco') == 4) |
        (F.col('3a dose') == 4) |
        (F.col('3o reforco') == 4) |
        (F.col('4a dose') == 4) |
        (F.col('5a dose') == 4) |
        (F.col('revacinacao') == 4) |
        (F.col('1a dose revacinacao') == 4) |
        (F.col('2a dose revacinacao') == 4) |
        (F.col('3a dose revacinacao') == 4) |
        (F.col('dose inicial') == 4) |
        (F.col('dose adicional') == 4) |
        (F.col('reforco') == 4) |
        (F.col('unica') == 4) |
        (F.col('tratamento com uma dose') == 4) |
        (F.col('tratamento com dezessete doses') == 4),
        F.lit(1)
    )
)

In [ ]:
vac = vac.withColumn(
    'num_ind_incom_delay_less2_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_incom_delay_2_6_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_incom_delay_more6_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [ ]:
vac = vac.withColumn(
    'num_dose_sec_uni_delay_less2_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_dose_sec_uni_delay_2_6_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_dose_sec_uni_delay_more6_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [ ]:
vac = vac.withColumn(
    'num_ind_com_atu_delay_less2_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_com_atu_delay_2_6_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_com_atu_delay_more6_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [ ]:
columns = doses.columns[22:]

In [ ]:
vac.groupby('date', 'code_mun_ibge').agg(
    *[F.count(col).alias(col) for col in new_columns]
).filter(F.col('date') >= '2020-01-01').orderBy('date') \
.write.csv(os.path.join(output_dir, 'vacinacao'), header=True, mode='overwrite')